# WX-AFD: Fine-Tuning Qwen3-4B for Area Forecast Discussions

This notebook fine-tunes **Qwen3-4B-Instruct-2507** with DoRA + rsLoRA to generate NWS Area Forecast Discussions from structured weather data.

**Pipeline context:** This is step 4 in the WX-AFD pipeline:
1. `01_scrape_afds.py` — Scrape AFDs from IEM
2. `02_fetch_weather.py` — Fetch weather data from Open-Meteo
3. `03_build_dataset.py` — Build training JSONL (messages format)
4. **`04_train.ipynb` — Fine-tune, evaluate, and export model** (this notebook)

---

**Table of Contents**
1. [Data Inspection](#1.-Data-Inspection)
2. [Configuration](#2.-Configuration)
3. [Sanity Check](#3.-Sanity-Check)
4. [Training](#4.-Training)
5. [Training Curves](#5.-Training-Curves)
6. [LoRA Merge](#6.-LoRA-Merge)
7. [Inference](#7.-Inference)
8. [Evaluation](#8.-Evaluation)
9. [Results](#9.-Results)
10. [Next Steps](#10.-Next-Steps)

In [ ]:
import json
import os
import subprocess
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# ---- Paths ----
# Adjust WX_AFD_ROOT if running outside Derecho
WX_AFD_ROOT = Path(os.environ.get(
    "WX_AFD_ROOT",
    f"/glade/derecho/scratch/{os.environ['USER']}/wx-afd"
))
DATA_DIR = WX_AFD_ROOT / "data"
CONFIG_PATH = WX_AFD_ROOT / "configs" / "wx-afd-dora.yml"
OUTPUT_DIR = WX_AFD_ROOT / "output"
EVAL_DIR = WX_AFD_ROOT / "eval"

MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"

# ---- GPU check ----
print(f"PyTorch:  {torch.__version__}")
print(f"CUDA:     {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU:      {torch.cuda.get_device_name(0)}")
    print(f"VRAM:     {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"Root:     {WX_AFD_ROOT}")

## 1. Data Inspection

Verify that our training data (output of `03_build_dataset.py`) is correctly formatted.

In [ ]:
def load_jsonl(path):
    """Load JSONL file into a list of dicts."""
    with open(path) as f:
        return [json.loads(line) for line in f]

train_data = load_jsonl(DATA_DIR / "train.jsonl")
val_data = load_jsonl(DATA_DIR / "val.jsonl")

print(f"Training examples:   {len(train_data)}")
print(f"Validation examples: {len(val_data)}")
print(f"Total:               {len(train_data) + len(val_data)}")

# Verify messages format
ex = train_data[0]
assert "messages" in ex, "Missing 'messages' key"
assert len(ex["messages"]) == 3, f"Expected 3 messages, got {len(ex['messages'])}"
assert ex["messages"][0]["role"] == "system"
assert ex["messages"][1]["role"] == "user"
assert ex["messages"][2]["role"] == "assistant"
print("\nMessages format: OK (system + user + assistant)")

In [ ]:
# Token length distribution
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

def count_tokens(example):
    text = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    return len(tokenizer.encode(text))

train_lengths = [count_tokens(ex) for ex in train_data]
val_lengths = [count_tokens(ex) for ex in val_data]
all_lengths = train_lengths + val_lengths

print(f"Token lengths (all {len(all_lengths)} examples):")
print(f"  Min:    {min(all_lengths)}")
print(f"  Mean:   {sum(all_lengths) // len(all_lengths)}")
print(f"  Median: {sorted(all_lengths)[len(all_lengths)//2]}")
print(f"  Max:    {max(all_lengths)}")
print(f"  >2048:  {sum(1 for l in all_lengths if l > 2048)} "
      f"({sum(1 for l in all_lengths if l > 2048)/len(all_lengths):.1%})")

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(all_lengths, bins=50, edgecolor="black", alpha=0.7)
ax.axvline(2048, color="red", linestyle="--", label="sequence_len=2048")
ax.set_xlabel("Total tokens per example")
ax.set_ylabel("Count")
ax.set_title("Token Length Distribution")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Display a sample training example
sample = train_data[0]
print("=" * 70)
print("SYSTEM PROMPT:")
print("=" * 70)
print(sample["messages"][0]["content"][:300], "...")
print()
print("=" * 70)
print("WEATHER INPUT (first 500 chars):")
print("=" * 70)
print(sample["messages"][1]["content"][:500], "...")
print()
print("=" * 70)
print("AFD OUTPUT (first 500 chars):")
print("=" * 70)
print(sample["messages"][2]["content"][:500], "...")

## 2. Configuration

Load and validate the Axolotl YAML config.

In [ ]:
import yaml

with open(CONFIG_PATH) as f:
    config = yaml.safe_load(f)

# Validate critical fields
checks = {
    "base_model": config.get("base_model") == "Qwen/Qwen3-4B-Instruct-2507",
    "chat_template": config.get("chat_template") == "qwen3",
    "eos_token": config.get("special_tokens", {}).get("eos_token") == "<|im_end|>",
    "pad_token": config.get("special_tokens", {}).get("pad_token") == "<|endoftext|>",
    "adapter": config.get("adapter") == "lora",
    "lora_r": config.get("lora_r") == 16,
    "lora_alpha": config.get("lora_alpha") == 32,
    "peft_use_dora": config.get("peft_use_dora") is True,
    "peft_use_rslora": config.get("peft_use_rslora") is True,
    "sample_packing": config.get("sample_packing") is True,
    "roles_to_train": config["datasets"][0].get("roles_to_train") == ["assistant"],
    "eot_tokens": config.get("eot_tokens") == ["<|im_end|>"],
    "test_datasets": "test_datasets" in config,
    "sequence_len": config.get("sequence_len") == 2048,
    "bf16": config.get("bf16") is True,
    "early_stopping": config.get("early_stopping_patience") == 5,
}

all_ok = True
for name, ok in checks.items():
    status = "OK" if ok else "FAIL"
    if not ok:
        all_ok = False
    print(f"  [{status}] {name}")

assert all_ok, "Config validation failed — fix issues above before training"
print("\nConfig validation: ALL PASSED")

## 3. Sanity Check

Use Axolotl's internals to validate config normalization, dataset loading, and loss masking
**before** launching a real training job. This catches issues early without GPU time.

Key checks:
- Config normalizes without errors
- Dataset loads and tokenizes correctly
- Labels are `-100` for system/user tokens (loss masking)
- EOS/PAD token IDs are correct

In [ ]:
from axolotl.utils.config import normalize_config, validate_config
from axolotl.utils.data import load_datasets
from axolotl.utils.dict import DictDefault

# Normalize and validate
cfg = DictDefault(config)
normalize_config(cfg)
validate_config(cfg)
print("Config normalization: OK")
print("Config validation:    OK")

# Load tokenizer for checks
tok = AutoTokenizer.from_pretrained(cfg.base_model, trust_remote_code=True)
eos_id = tok.convert_tokens_to_ids("<|im_end|>")
pad_id = tok.convert_tokens_to_ids("<|endoftext|>")
print(f"EOS token ID: {eos_id} (expect 151645) — {'OK' if eos_id == 151645 else 'FAIL'}")
print(f"PAD token ID: {pad_id} (expect 151643) — {'OK' if pad_id == 151643 else 'FAIL'}")
assert eos_id != pad_id, "EOS and PAD must differ!"

# Load datasets (this tests the full data pipeline)
print("\nLoading datasets...")
train_dataset, eval_dataset, _, _ = load_datasets(cfg=cfg, cli_args={})
print(f"  Train: {len(train_dataset)} packed sequences")
if eval_dataset:
    print(f"  Eval:  {len(eval_dataset)} packed sequences")

# Verify loss masking: labels should be -100 for system/user tokens
sample = train_dataset[0]
labels = sample["labels"]
n_masked = sum(1 for l in labels if l == -100)
n_total = len(labels)
print(f"\nLoss masking (sample 0):")
print(f"  Total tokens:  {n_total}")
print(f"  Masked (-100): {n_masked} ({n_masked/n_total:.1%})")
print(f"  Trained:       {n_total - n_masked} ({(n_total - n_masked)/n_total:.1%})")
assert n_masked > 0, "No masked tokens — loss masking may be broken"
print("\nSanity check: ALL PASSED")

## 4. Training

Run training directly in this notebook session (requires GPU compute node).

**Expected timeline:** ~429 steps across 3 epochs, ~45-75 minutes on A100 40GB.
Early stopping (patience=5) may terminate after epoch 2.

In [ ]:
# INTERACTIVE MODE: Run training directly
# Only run this cell if you're on a compute node with a GPU

from axolotl.train import train

train(cfg=cfg)

In [ ]:
# Monitor training logs
log_dir = WX_AFD_ROOT / "logs"
logs = sorted(log_dir.glob("train*.log")) if log_dir.exists() else []
if logs:
    latest = logs[-1]
    print(f"\nLatest log: {latest}")
    print("--- Last 20 lines ---")
    with open(latest) as f:
        lines = f.readlines()
        for line in lines[-20:]:
            print(line, end="")
else:
    print("No training logs found yet.")

In [ ]:
# Find trainer_state.json
state_files = sorted(OUTPUT_DIR.glob("**/trainer_state.json"))
if not state_files:
    print("No trainer_state.json found — training may not have completed yet.")
else:
    state_path = state_files[-1]
    print(f"Loading: {state_path}")
    state = json.loads(state_path.read_text())

    # Extract metrics
    train_loss, train_steps = [], []
    eval_loss, eval_steps = [], []
    lr_values, lr_steps = [], []

    for entry in state["log_history"]:
        step = entry["step"]
        if "loss" in entry:
            train_loss.append(entry["loss"])
            train_steps.append(step)
        if "eval_loss" in entry:
            eval_loss.append(entry["eval_loss"])
            eval_steps.append(step)
        if "learning_rate" in entry:
            lr_values.append(entry["learning_rate"])
            lr_steps.append(step)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Loss curves
    ax1.plot(train_steps, train_loss, label="Train Loss", alpha=0.7)
    if eval_loss:
        ax1.plot(eval_steps, eval_loss, label="Val Loss", marker="o", markersize=4)
    ax1.set_xlabel("Step")
    ax1.set_ylabel("Loss")
    ax1.set_title("Training & Validation Loss")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # LR schedule
    if lr_values:
        ax2.plot(lr_steps, lr_values, color="green")
        ax2.set_xlabel("Step")
        ax2.set_ylabel("Learning Rate")
        ax2.set_title("Learning Rate Schedule")
        ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Summary
    print(f"\nTotal steps:     {state['global_step']}")
    print(f"Best model step: {state.get('best_model_checkpoint', 'N/A')}")
    if eval_loss:
        print(f"Best val loss:   {min(eval_loss):.4f} (step {eval_steps[eval_loss.index(min(eval_loss))]})")
    print(f"Final train loss: {train_loss[-1]:.4f}")

## 6. LoRA Merge

Merge the LoRA adapter back into the base model for clean inference.
The merged model is a standard HuggingFace directory — no adapter loading needed.

In [ ]:
# Merge LoRA adapter into base model
merge_cmd = [
    "accelerate", "launch", "-m", "axolotl.cli.merge_lora",
    str(CONFIG_PATH),
    "--lora_model_dir", str(OUTPUT_DIR),
]
print(f"Running: {' '.join(merge_cmd)}")
result = subprocess.run(merge_cmd, capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print(f"STDERR:\n{result.stderr}")
    raise RuntimeError("Merge failed")

merged_dir = OUTPUT_DIR / "merged"
assert merged_dir.exists(), f"Merged model not found at {merged_dir}"
print(f"\nMerged model saved to: {merged_dir}")
print(f"Contents: {[f.name for f in sorted(merged_dir.iterdir())]}")

In [ ]:
# Copy to persistent storage
persist_dir = Path(f"/glade/work/{os.environ['USER']}/wx-afd-model-v1")
persist_dir.mkdir(parents=True, exist_ok=True)

import shutil
for item in merged_dir.iterdir():
    dst = persist_dir / item.name
    if item.is_file() and not dst.exists():
        shutil.copy2(item, dst)
    elif item.is_dir() and not dst.exists():
        shutil.copytree(item, dst)

print(f"Merged model copied to: {persist_dir}")

## 7. Inference

Load the merged model and generate AFDs. Compare against reference forecasts.

In [ ]:
from wx_afd import generate_afd, load_model

# Load merged model
merged_dir = OUTPUT_DIR / "merged"
print(f"Loading merged model from: {merged_dir}")
model, tokenizer = load_model(str(merged_dir))
print("Model loaded.")

# Generate from a validation example
val_ex = val_data[0]
weather_input = val_ex["messages"][1]["content"]
reference_afd = val_ex["messages"][2]["content"]

generated_afd = generate_afd(model, tokenizer, weather_input)

print("=" * 70)
print("GENERATED AFD:")
print("=" * 70)
print(generated_afd[:1000], "..." if len(generated_afd) > 1000 else "")
print()
print("=" * 70)
print("REFERENCE AFD:")
print("=" * 70)
print(reference_afd[:1000], "..." if len(reference_afd) > 1000 else "")

In [ ]:
# EOS behavior check: verify model stops generating
print("EOS Behavior Check — generating 5 examples...")
print()
for i in range(min(5, len(val_data))):
    inp = val_data[i]["messages"][1]["content"]
    out = generate_afd(model, tokenizer, inp)
    terminated = len(out) < 2048 * 4  # rough char-level check
    print(f"  Example {i}: {len(out)} chars — {'OK (terminated)' if terminated else 'WARNING: may not have stopped'}")

print("\nAll examples should terminate cleanly with <|im_end|>.")

## 8. Evaluation

Full evaluation on all validation examples using ROUGE-1/2/L, BERTScore F1, and format compliance.

In [ ]:
from rouge_score import rouge_scorer
from tqdm.notebook import tqdm
from wx_afd import REQUIRED_SECTIONS, compute_rouge, compute_bertscore, format_compliance

# Generate all predictions
predictions = []
references = []

print(f"Generating AFDs for {len(val_data)} validation examples...")
for ex in tqdm(val_data):
    pred = generate_afd(model, tokenizer, ex["messages"][1]["content"])
    predictions.append(pred)
    references.append(ex["messages"][2]["content"])

# Save generated AFDs
ft_eval_dir = EVAL_DIR / "finetuned"
ft_gen_dir = ft_eval_dir / "generated"
ft_scores_dir = ft_eval_dir / "scores"
ft_gen_dir.mkdir(parents=True, exist_ok=True)
ft_scores_dir.mkdir(parents=True, exist_ok=True)

for i, pred in enumerate(predictions):
    (ft_gen_dir / f"example_{i:04d}.txt").write_text(pred)

# ROUGE
rouge_avg = compute_rouge(predictions, references)
print(f"\nROUGE-1: {rouge_avg['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_avg['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_avg['rougeL']:.4f}")

# Per-example ROUGE (for later analysis)
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_results = {"rouge1": [], "rouge2": [], "rougeL": []}
for pred, ref in zip(predictions, references):
    result = scorer.score(ref, pred)
    for key in rouge_results:
        rouge_results[key].append(result[key].fmeasure)

# Free model memory before BERTScore
del model
torch.cuda.empty_cache()

# BERTScore
print("\nComputing BERTScore (this may take a minute)...")
bertscore_f1 = compute_bertscore(predictions, references)
print(f"BERTScore F1: {bertscore_f1:.4f}")

# Format compliance
compliance = [format_compliance(pred)["compliance_score"] for pred in predictions]
avg_compliance = sum(compliance) / len(compliance)
print(f"Format compliance: {avg_compliance:.2%}")

# Save metrics
ft_metrics = {
    "tag": "finetuned",
    "num_examples": len(predictions),
    "rouge1": rouge_avg["rouge1"],
    "rouge2": rouge_avg["rouge2"],
    "rougeL": rouge_avg["rougeL"],
    "bertscore_f1": bertscore_f1,
    "format_compliance": avg_compliance,
}
with open(ft_scores_dir / "metrics.json", "w") as f:
    json.dump(ft_metrics, f, indent=2)
print(f"\nMetrics saved to {ft_scores_dir / 'metrics.json'}")

In [ ]:
# Zero-shot baseline evaluation
print(f"Loading zero-shot baseline: {MODEL_ID}")
model_zs, tokenizer_zs = load_model(MODEL_ID)

zs_predictions = []
print(f"Generating zero-shot AFDs for {len(val_data)} examples...")
for ex in tqdm(val_data):
    pred = generate_afd(model_zs, tokenizer_zs, ex["messages"][1]["content"])
    zs_predictions.append(pred)

# Save
zs_eval_dir = EVAL_DIR / "zero-shot"
zs_gen_dir = zs_eval_dir / "generated"
zs_scores_dir = zs_eval_dir / "scores"
zs_gen_dir.mkdir(parents=True, exist_ok=True)
zs_scores_dir.mkdir(parents=True, exist_ok=True)
for i, pred in enumerate(zs_predictions):
    (zs_gen_dir / f"example_{i:04d}.txt").write_text(pred)

# ROUGE
zs_rouge_avg = compute_rouge(zs_predictions, references)

# Free model before BERTScore
del model_zs
torch.cuda.empty_cache()

# BERTScore
print("Computing BERTScore for zero-shot...")
zs_bertscore = compute_bertscore(zs_predictions, references)

# Compliance
zs_compliance = [format_compliance(pred)["compliance_score"] for pred in zs_predictions]
zs_avg_compliance = sum(zs_compliance) / len(zs_compliance)

zs_metrics = {
    "tag": "zero-shot",
    "num_examples": len(zs_predictions),
    "rouge1": zs_rouge_avg["rouge1"],
    "rouge2": zs_rouge_avg["rouge2"],
    "rougeL": zs_rouge_avg["rougeL"],
    "bertscore_f1": zs_bertscore,
    "format_compliance": zs_avg_compliance,
}
with open(zs_scores_dir / "metrics.json", "w") as f:
    json.dump(zs_metrics, f, indent=2)

print(f"\nZero-shot ROUGE-1: {zs_rouge_avg['rouge1']:.4f}")
print(f"Zero-shot ROUGE-L: {zs_rouge_avg['rougeL']:.4f}")
print(f"Zero-shot BERTScore F1: {zs_bertscore:.4f}")
print(f"Zero-shot compliance: {zs_avg_compliance:.2%}")

In [ ]:
# AFD format compliance detail
print("AFD Section Presence (fine-tuned model):")
print()
for sec in REQUIRED_SECTIONS:
    present = sum(1 for p in predictions if sec in p.lower())
    print(f"  {sec:<15} {present}/{len(predictions)} ({present/len(predictions):.0%})")

print("\nAFD Section Presence (zero-shot baseline):")
print()
for sec in REQUIRED_SECTIONS:
    present = sum(1 for p in zs_predictions if sec in p.lower())
    print(f"  {sec:<15} {present}/{len(zs_predictions)} ({present/len(zs_predictions):.0%})")

## 9. Results

Compare fine-tuned model against zero-shot baseline.

In [ ]:
# Comparison table
print(f"{'Metric':<20} {'Fine-tuned':>12} {'Zero-shot':>12} {'Delta':>12}")
print("-" * 58)
for k in ["rouge1", "rouge2", "rougeL", "bertscore_f1", "format_compliance"]:
    f_val = ft_metrics[k]
    z_val = zs_metrics[k]
    delta = f_val - z_val
    sign = "+" if delta > 0 else ""
    print(f"{k:<20} {f_val:>12.4f} {z_val:>12.4f} {sign}{delta:>11.4f}")

# Bar chart
import numpy as np

metrics_keys = ["rouge1", "rouge2", "rougeL", "bertscore_f1", "format_compliance"]
labels = ["ROUGE-1", "ROUGE-2", "ROUGE-L", "BERTScore\nF1", "Format\nCompliance"]
ft_vals = [ft_metrics[k] for k in metrics_keys]
zs_vals = [zs_metrics[k] for k in metrics_keys]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 5))
bars1 = ax.bar(x - width/2, ft_vals, width, label="Fine-tuned", color="#2196F3")
bars2 = ax.bar(x + width/2, zs_vals, width, label="Zero-shot", color="#FF9800")

ax.set_ylabel("Score")
ax.set_title("Fine-tuned vs Zero-shot Evaluation")
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
ax.set_ylim(0, 1)
ax.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Best and worst 3 examples by ROUGE-L
rougeL_scores = rouge_results["rougeL"]
indexed = sorted(enumerate(rougeL_scores), key=lambda x: x[1], reverse=True)

print("=" * 70)
print("TOP 3 (Best ROUGE-L):")
print("=" * 70)
for rank, (idx, score) in enumerate(indexed[:3], 1):
    print(f"\n--- #{rank}: Example {idx} (ROUGE-L = {score:.4f}) ---")
    print("GENERATED (first 300 chars):")
    print(predictions[idx][:300])
    print()

print("\n" + "=" * 70)
print("BOTTOM 3 (Worst ROUGE-L):")
print("=" * 70)
for rank, (idx, score) in enumerate(indexed[-3:], 1):
    print(f"\n--- #{rank}: Example {idx} (ROUGE-L = {score:.4f}) ---")
    print("GENERATED (first 300 chars):")
    print(predictions[idx][:300])
    print("REFERENCE (first 300 chars):")
    print(references[idx][:300])
    print()

## 10. Next Steps

**Completed in this notebook:**
- Data inspection and validation
- Config verification with all corrections applied
- Sanity check (loss masking, token IDs, dataset loading)
- Model training (DoRA + rsLoRA, 3 epochs with early stopping)
- LoRA merge and export to persistent storage
- Full evaluation: ROUGE, BERTScore, format compliance
- Zero-shot baseline comparison

**Next iterations (out of scope for v1):**
- Multi-WFO training (expand beyond Louisville)
- AlignScore factual consistency evaluation
- GRPO/DPO alignment with AlignScore as reward signal
- Attribute-specific LoRA adapters (synopsis vs aviation)
- vLLM/TGI deployment for real-time inference

**Model location:** `/glade/work/$USER/wx-afd-model-v1/`